In [1]:
# Import libraries
from gector.gec_model import GecBERTModel

# Create an instance of the model
model = GecBERTModel(vocab_path = "./data/output_vocabulary", model_paths = ["./gector/roberta-large_1_pie_1bw_st3.th"])






/home/ronit/miniconda3/envs/gector_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ronit/miniconda3/envs/gector_env/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [24]:

# Add the sentence with grammatical errors
sent = 'hey guys hows it going my names george'

In [25]:
# Create an empty list to store the words 
batch = []
batch.append(sent.split())
#temp printing batch-teh list of sentence with errors
for i in range(len(batch)):
    print(batch[i])
#temp printing final batch - the list of corrected sentence  
final_batch, total_updates = model.handle_batch(batch)
#temp
for i in range(len(final_batch)):
    print(final_batch[i])
    print(total_updates)
#temp
updated_sent = " ".join(final_batch[0])

['hey', 'guys', 'hows', 'it', 'going', 'my', 'names', 'george']
['Hey', 'guys', ',', 'how', 'is', 'it', 'going', '?', 'My', 'name', 'is', 'George']
3


In [32]:
def generate_output_dict(input_sentence, corrected_sentence):
    output_dict_list = []
    end_index = -2
    i = 0
    j = i
    while i < len(input_sentence) and j<len(corrected_sentence):
        original_word = input_sentence[i]
        corrected_word = corrected_sentence[j]

        if original_word.lower() == corrected_word.lower():
            start_index = end_index + 2
            end_index = start_index
            end_index += len(corrected_word) 
            output_dict = {
                'word': original_word,
                'correction': corrected_word,
                'start_index': start_index,
                'end_index': end_index,
            }
            if(original_word != corrected_word):
                output_dict_list.append(output_dict)
            i += 1
            j += 1
            continue
        
        # Check up to 3 tokens in the corrected sentence for correction
        correction_tokens = []
        k = j+2
        while j < min(k, len(corrected_sentence)):
            correction_tokens.append(corrected_sentence[j])
            if original_word.lower() == corrected_sentence[j].lower():
                j += 1
                break
            j += 1
            

        correction = ' '.join(correction_tokens) if any(correction_tokens) else None

        start_index = end_index
        end_index = start_index + len(correction)

        output_dict = {
            'word': original_word,
            'correction': correction,
            'start_index': start_index,
            'end_index': end_index,
        }
        output_dict_list.append(output_dict)

        i += 1

    # while(i<len(input_sentence)):
        
    return output_dict_list



In [33]:
# input_sentence = ['hey', 'guys', 'hows', 'it', 'going', 'my', 'names', 'george']
# corrected_sentence = ['Hey', 'guys', ',', 'how', 'is', 'it', 'going', '?', 'My', 'name', 'is', 'George']

output_dict_list = generate_output_dict(batch[0], final_batch[0])

# Print the output
for output_dict in output_dict_list:
    print(output_dict)


{'word': 'hey', 'correction': 'Hey', 'start_index': 0, 'end_index': 3}
{'word': 'hows', 'correction': ', how', 'start_index': 9, 'end_index': 14}
{'word': 'it', 'correction': 'is it', 'start_index': 14, 'end_index': 19}
{'word': 'my', 'correction': '? My', 'start_index': 26, 'end_index': 30}
{'word': 'names', 'correction': 'name is', 'start_index': 30, 'end_index': 37}
{'word': 'george', 'correction': 'George', 'start_index': 39, 'end_index': 45}


In [6]:
print(f"Original Sentence: {sent}\n")
print(f"Updated Sentence: {updated_sent}")

Original Sentence: hey guys hows it going my names george

Updated Sentence: Hey guys , how is it going ? My name is George
